In [ ]:
import matplotlib.pyplot as plt
import ee
import joblib
import geemap
from matplotlib.colors import ListedColormap

In [ ]:
# Authenticate and initialize the Earth Engine API
ee.Authenticate()
ee.Initialize(project='svevazanettieo1') #così apro il progetto esistente in GEE, non ne sto creando uno nuovo

In [ ]:
# Load model 
clf = joblib.load("random_forest_model_train+val.pkl")

In [ ]:
sentinel2018 = ee.Image('projects/svevazanettieo1/assets/Dubai/sentinel2018')
sentinel2020 = ee.Image('projects/svevazanettieo1/assets/Dubai/sentinel2020')
sentinel2022 = ee.Image('projects/svevazanettieo1/assets/Dubai/sentinel2022')
sentinel2024 = ee.Image('projects/svevazanettieo1/assets/Dubai/sentinel2024')

AOI = ee.FeatureCollection('projects/svevazanettieo1/assets/Dubai/Area_of_interest_Dubai')

In [ ]:
# Convert the FeatureCollection to a GeoDataFrame
# Sample pixels as a FeatureCollection
sample2018 = sentinel2018.sample(
    region=AOI,
    scale=10,
    numPixels=1000000,
    geometries=True
)

gdf2018 = geemap.ee_to_gdf(sample2018)

print(gdf2018.head())

In [ ]:
sample2020 = sentinel2020.sample(
    region=AOI,
    scale=10,
    numPixels=1000000,
    geometries=True
)

gdf2020 = geemap.ee_to_gdf(sample2020)

print(gdf2020.head())

In [ ]:
sample2022 = sentinel2022.sample(
    region=AOI,
    scale=10,
    numPixels=1000000,
    geometries=True
)

gdf2022 = geemap.ee_to_gdf(sample2022)

print(gdf2022.head())

In [ ]:
sample2024 = sentinel2024.sample(
    region=AOI,
    scale=10,
    numPixels=1000000,
    geometries=True
)

gdf2024 = geemap.ee_to_gdf(sample2024)

print(gdf2024.head())

In [ ]:
gdf2018.to_file("points_2018.geojson", driver="GeoJSON")
gdf2020.to_file("points_2020.geojson", driver="GeoJSON")
gdf2022.to_file("points_2022.geojson", driver="GeoJSON")
gdf2024.to_file("points_2024.geojson", driver="GeoJSON")

In [ ]:
def preprocess(gdf):
    gdf.columns = [col.replace('B', 'properties.B') for col in gdf.columns]
    gdf.columns = [col.replace('n', 'properties.n') for col in gdf.columns]
    feature_columns = [col for col in gdf.columns if col.startswith('properties.B') or col.startswith('properties.n')]
    whole_image = gdf[feature_columns]
    return whole_image

In [ ]:
image2018 = preprocess(gdf2018)
image2020 = preprocess(gdf2020)
image2022 = preprocess(gdf2022)
image2024 = preprocess(gdf2024)

In [ ]:
scaler = joblib.load("scaler.save")

image2018 = scaler.transform(image2018)
image2020 = scaler.transform(image2020)
image2022 = scaler.transform(image2022)
image2024 = scaler.transform(image2024)

In [ ]:
# Modello precedentemente allenato
y_pred_2018 = clf.predict(image2018)
gdf2018['predicted_class'] = y_pred_2018

y_pred_2020 = clf.predict(image2020)
gdf2020['predicted_class'] = y_pred_2020

y_pred_2022 = clf.predict(image2022)
gdf2022['predicted_class'] = y_pred_2022

y_pred_2024 = clf.predict(image2024)
gdf2024['predicted_class'] = y_pred_2024


In [ ]:


# plot con colormap discreta
ax = gdf.plot(column='predicted_class', cmap=cmap, legend=True, categorical=True)
ax.set_axis_off()
plt.show()

In [ ]:
# definisco la lista di colori nell'ordine delle classi
colors = ['green', 'cyan', 'yellow', 'red']  

# creo la colormap discreta
cmap = ListedColormap(colors)

fig, axes = plt.subplots(2, 2, figsize=(10, 10))  # griglia 2x2
axes = axes.flatten()  # trasformo in lista per scorrere facilmente

title = ["2018", "2020", "2022", "2024"]
gdf=[gdf2018, gdf2020, gdf2022, gdf2024]

for i in range(len(title)):
    gdf[i].plot(
        column='predicted_class',
        cmap=cmap,
        legend=True,
        categorical=True,
        ax=axes[i]
    )
    axes[i].set_title(title[i])
    axes[i].set_axis_off()

plt.tight_layout()
plt.show()
